# Imports and Data

In [2]:
import pandas as pd
import requests
import time
import json
import numpy as np


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("detach", True)
ser = Service("C:\Program Files (x86)\chromedriver.exe")


source_data = pd.read_excel("../../Data/ERA_Alabama.xlsx",sheet_name="All Plants")

In [3]:
scientific_names = source_data["Scientific Name"].to_list()

# Find which ERA Plants have pages on the site 

In [4]:
scientific_names

['Acacia farnesiana',
 'Acalypha rhomboidea',
 'Acalypha virginica',
 'Acer floridanum',
 'Acer negundo',
 'Acer nigrum',
 'Acer rubrum',
 'Acer saccharinum',
 'Acer saccharum',
 'Acer spicatum',
 'Achillea millefolium',
 'Acmella oppositifolia',
 'Aconitum uncinatum',
 'Actaea pachypoda',
 'Actaea racemosa',
 'Adiantum capillus-veneris',
 'Adiantum pedatum',
 'Aeschynomene americana',
 'Aeschynomene viscidula',
 'Aesculus flava',
 'Aesculus glabra',
 'Aesculus parviflora',
 'Aesculus pavia',
 'Aesculus sylvatica',
 'Agalinis auriculata',
 'Agalinis maritima',
 'Agalinis purpurea',
 'Agalinis tenuifolia',
 'Agastache nepetoides',
 'Ageratina altissima',
 'Agrimonia gryposepala',
 'Agrimonia parviflora',
 'Agrimonia pubescens',
 'Agrimonia rostellata',
 'Agrimonia striata',
 'Agrostis hyemalis',
 'Agrostis perennans',
 'Agrostis scabra',
 'Alisma subcordatum',
 'Allium canadense',
 'Allium cernuum',
 'Allium tricoccum',
 'Alnus serrulata',
 'Alopecurus carolinianus',
 'Amaranthus hybrid

In [5]:
matches = []

In [7]:
import string

driver = webdriver.Chrome(service=ser, options=options)

for letter in string.ascii_uppercase:

    print(letter)

    driver.get(f"https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderListResults.aspx?letter={letter}")


    outer_div = driver.find_element(By.CLASS_NAME,"p2")
    link_elements = outer_div.find_elements(By.CSS_SELECTOR,"div div a")
    names = [el.text.split("'")[0] for el in link_elements]
    links = [el.get_attribute("href") for el in link_elements]
    all_name_link_pairs = list(set(list(zip(names,links))))
    all_name_link_pairs.sort()

    for pair in all_name_link_pairs:
        if pair[0] in scientific_names:
            print(pair)
            matches.append(pair)

    

driver.close()


with open("Matches.txt","w") as f:
    for match in matches:
        f.write(str(match) + "\n")
    


# Scrape 1 

In [14]:
driver = webdriver.Chrome(service=ser, options=options)
driver.get(f"https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=279992&isprofile=0&letter=V")

row_elements = driver.find_elements(By.CSS_SELECTOR,"div.column-right div.row")
plant_dict = {"name":{}}
for el in row_elements[:-1]:
    
    temp = el.text.split(":")
    plant_dict["name"].update({temp[0]:temp[1]})

print(plant_dict)

driver.close()


{'name': {'Common Name': ' highbush blueberry ', 'Type': ' Deciduous shrub', 'Family': ' Ericaceae', 'Native Range': ' North America', 'Zone': ' 5 to 8', 'Height': ' 6.00 to 12.00 feet', 'Spread': ' 8.00 to 12.00 feet', 'Bloom Time': ' May', 'Bloom Description': ' White or pinkish', 'Sun': ' Full sun to part shade', 'Water': ' Medium to wet', 'Maintenance': ' Medium', 'Suggested Use': ' Hedge', 'Flower': ' Showy', 'Leaf': ' Good Fall', 'Attracts': ' Birds, Butterflies', 'Fruit': ' Showy, Edible', 'Tolerate': ' Wet Soil'}}


# Scrape All

In [15]:
with open("Matches.txt") as f:
    matches = [eval(x) for x in f.readlines()]

[('Acer negundo',
  'https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=275364&isprofile=0&letter=A'),
 ('Acer rubrum',
  'https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=275374&isprofile=0&letter=A'),
 ('Acer saccharinum',
  'https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=275381&isprofile=0&letter=A'),
 ('Acer saccharum',
  'https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=275382&isprofile=0&letter=A'),
 ('Achillea millefolium',
  'https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=277129&isprofile=0&letter=A')]

In [24]:
driver = webdriver.Chrome(service=ser, options=options)

all_plant_dicts = {}

for name,link in matches:
    print(name)
    driver.get(link)

    row_elements = driver.find_elements(By.CSS_SELECTOR,"div.column-right div.row")
    plant_dict = {name:{}}
    for el in row_elements[:-1]:
        column,value = el.text.split(":")
        plant_dict[name].update({column:value})
    all_plant_dicts.update(plant_dict)



    

Acer negundo
Acer rubrum
Acer saccharinum
Acer saccharum
Achillea millefolium
Actaea pachypoda
Actaea racemosa
Adiantum capillus-veneris
Adiantum pedatum
Aesculus flava
Aesculus glabra
Aesculus parviflora
Aesculus pavia
Aesculus sylvatica
Agalinis tenuifolia
Agastache nepetoides
Ageratina altissima
Allium cernuum
Alnus serrulata
Amelanchier arborea
Amelanchier canadensis
Amelanchier laevis
Amelanchier obovalis
Amelanchier stolonifera
Amorpha fruticosa
Amsonia ciliata
Amsonia tabernaemontana
Andropogon gerardii
Andropogon glomeratus
Antennaria parlinii
Antennaria plantaginifolia
Apocynum cannabinum
Aquilegia canadensis
Aralia racemosa
Aralia spinosa
Arisaema dracontium
Arisaema triphyllum
Aristolochia macrophylla
Aristolochia tomentosa
Arnoglossum atriplicifolium
Arnoglossum reniforme
Aronia arbutifolia
Aronia melanocarpa
Aruncus dioicus
Arundinaria gigantea
Asarum canadense
Asclepias hirtella
Asclepias incarnata
Asclepias quadrifolia
Asclepias syriaca
Asclepias tuberosa
Asclepias verti

In [25]:
pd.DataFrame(all_plant_dicts).T.to_csv("./Output/MissouriBotanical_Unedited.csv")